In [1]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
connection = sqlite3.connect('sqlite_db_pythonsqlite.db')
cursor = connection.cursor()

print(f"Connected to SQLite version: {sqlite3.version}")

Connected to SQLite version: 2.6.0


### Q1: Some of the facilities charge a fee to members, but some do not. Write a SQL query to produce a list of the names of the facilities that do.

In [2]:
# SQL query to retrieve all members who joined after 2020-01-01
cursor.execute("SELECT name FROM Facilities WHERE membercost > 0.0;")
df = pd.DataFrame(cursor.fetchall())

df.columns = [description[0] for description in cursor.description]

# Display the results
print(df.head())



             name
0  Tennis Court 1
1  Tennis Court 2
2  Massage Room 1
3  Massage Room 2
4    Squash Court


### Q2: How many facilities do not charge a fee to members? 

In [3]:
# Count the number of facilities with no member cost
cursor.execute("SELECT COUNT(name) AS num_facilites FROM Facilities WHERE membercost = 0.0;")
df = pd.DataFrame(cursor.fetchall())

#Display column names
df.columns = [description[0] for description in cursor.description]

# Display the results
print(df.head())



   num_facilites
0              4


### Q3: Write an SQL query to show a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost. Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [4]:
# Retrieve facilities charging less than 20% of monthly maintenance cost
cursor.execute("SELECT name, membercost, monthlymaintenance FROM Facilities WHERE membercost > 0.0 AND membercost < 0.2 * monthlymaintenance;")
df = pd.DataFrame(cursor.fetchall())

#Display column names
df.columns = [description[0] for description in cursor.description]

# Display the results
print(df.head())



             name  membercost  monthlymaintenance
0  Tennis Court 1         5.0                 200
1  Tennis Court 2         5.0                 200
2  Massage Room 1         9.9                3000
3  Massage Room 2         9.9                3000
4    Squash Court         3.5                  80


### Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5. Try writing the query without using the OR operator.

In [5]:
# Retrieve details for facilities with IDs 1 and 5 using IN
cursor.execute("SELECT * FROM Facilities WHERE facid IN (1, 5);")
df = pd.DataFrame(cursor.fetchall())

#Display column names
df.columns = [description[0] for description in cursor.description]

# Display the results
print(df.head())



   facid            name  membercost  guestcost  initialoutlay  \
0      1  Tennis Court 2         5.0         25           8000   
1      5  Massage Room 2         9.9         80           4000   

   monthlymaintenance  
0                 200  
1                3000  


### Q5: Produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100. Return the name and monthly maintenance of the facilities in question.

In [6]:
# Label facilities based on monthly maintenance cost
cursor.execute("SELECT name, monthlymaintenance, CASE WHEN monthlymaintenance > 100 THEN 'expensive' ELSE 'cheap' END AS cost_label FROM Facilities;")
df = pd.DataFrame(cursor.fetchall())

#Display column names
df.columns = [description[0] for description in cursor.description]

# Display the results
print(df.head())



              name  monthlymaintenance cost_label
0   Tennis Court 1                 200  expensive
1   Tennis Court 2                 200  expensive
2  Badminton Court                  50      cheap
3     Table Tennis                  10      cheap
4   Massage Room 1                3000  expensive


### Q6: You'd like to get the first and last name of the last member(s) who signed up. Try not to use the LIMIT clause for your solution.

In [7]:
# Retrieve first and last names of the last member(s) who signed up
cursor.execute("SELECT firstname, surname FROM Members ORDER BY joindate ASC;")
df = pd.DataFrame(cursor.fetchall())

#Display column names
df.columns = [description[0] for description in cursor.description]

# Display the results
print(df.head())



  firstname   surname
0     GUEST     GUEST
1    Darren     Smith
2     Tracy     Smith
3       Tim    Rownam
4    Janice  Joplette


### Q7: Produce a list of all members who have used a tennis court. Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name.

In [8]:
# Retrieve members who have used a tennis court
cursor.execute("SELECT DISTINCT f.name, m.firstname||' '||m.surname FROM Facilities AS f JOIN Members AS m ON f.facid = m.memid WHERE f.name LIKE 'Tennis%'  ORDER BY m.firstname;")
df = pd.DataFrame(cursor.fetchall())

#Display column names
df.columns = [description[0] for description in cursor.description]

# Display the results
print(df.head())



             name m.firstname||' '||m.surname
0  Tennis Court 2                Darren Smith
1  Tennis Court 1                 GUEST GUEST


### Q8: Produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30. Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.

In [9]:
# Retrieve bookings on 2012-09-14 with costs greater than $30
cursor.execute("SELECT  Facilities.name AS facility_name, Members.firstname || ' ' || Members.surname AS member_name, CASE  WHEN Bookings.memid = 0 THEN Bookings.slots * Facilities.guestcost ELSE Bookings.slots * Facilities.membercost END AS cost FROM Bookings JOIN Facilities ON Bookings.facid = Facilities.facid JOIN Members ON Bookings.memid = Members.memid WHERE DATE(Bookings.starttime) = '2012-09-14' AND CASE WHEN Bookings.memid = 0 THEN Bookings.slots * Facilities.guestcost ELSE Bookings.slots * Facilities.membercost END > 30 ORDER BY cost DESC;")               
df = pd.DataFrame(cursor.fetchall())

#Display column names
df.columns = [description[0] for description in cursor.description]

# Display the results
print(df.head())


    facility_name  member_name   cost
0  Massage Room 2  GUEST GUEST  320.0
1  Massage Room 1  GUEST GUEST  160.0
2  Massage Room 1  GUEST GUEST  160.0
3  Massage Room 1  GUEST GUEST  160.0
4  Tennis Court 2  GUEST GUEST  150.0


### Q9: This time, produce the same result as in Q8, but using a subquery.

In [10]:
# Retrieve bookings on 2012-09-14 with costs greater than $30 using a subquery
cursor.execute("SELECT Facilities.name AS facility_name, Members.firstname || ' ' || Members.surname AS member_name, CASE WHEN Bookings.memid = 0 THEN Bookings.slots * Facilities.guestcost ELSE Bookings.slots * Facilities.membercost END AS cost FROM Bookings JOIN Facilities ON Bookings.facid = Facilities.facid JOIN Members ON Bookings.memid = Members.memid WHERE DATE(Bookings.starttime) = '2012-09-14' AND CASE WHEN Bookings.memid = 0 THEN Bookings.slots * Facilities.guestcost ELSE Bookings.slots * Facilities.membercost END > 30 ORDER BY cost DESC;") 
df = pd.DataFrame(cursor.fetchall())

#Display column names
df.columns = [description[0] for description in cursor.description]

# Display the results
print(df.head())



    facility_name  member_name   cost
0  Massage Room 2  GUEST GUEST  320.0
1  Massage Room 1  GUEST GUEST  160.0
2  Massage Room 1  GUEST GUEST  160.0
3  Massage Room 1  GUEST GUEST  160.0
4  Tennis Court 2  GUEST GUEST  150.0


### Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [11]:
# Retrieve facilities with total revenue less than $1000
cursor.execute("SELECT Facilities.name AS facility_name, SUM(CASE WHEN Bookings.memid = 0 THEN Bookings.slots * Facilities.guestcost ELSE Bookings.slots * Facilities.membercost END) AS total_revenue FROM Bookings JOIN Facilities ON Bookings.facid = Facilities.facid GROUP BY Facilities.name HAVING total_revenue < 1000 ORDER BY total_revenue;")
df = pd.DataFrame(cursor.fetchall())

#Display column names
df.columns = [description[0] for description in cursor.description]

# Display the results
print(df.head())



   facility_name  total_revenue
0   Table Tennis            180
1  Snooker Table            240
2     Pool Table            270


### Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [12]:
# Retrieve members and who recommended them
cursor.execute("SELECT m1.firstname || ' ' || m1.surname AS member_name, m2.firstname || ' ' || m2.surname AS recommended_by FROM Members m1 LEFT JOIN Members m2 ON m1.recommendedby = m2.memid ORDER BY m1.surname, m1.firstname;")
df = pd.DataFrame(cursor.fetchall())

#Display column names
df.columns = [description[0] for description in cursor.description]

# Display the results
print(df.head())



      member_name   recommended_by
0  Florence Bader  Ponder Stibbons
1      Anne Baker  Ponder Stibbons
2   Timothy Baker   Jemima Farrell
3      Tim Boothe       Tim Rownam
4  Gerald Butters     Darren Smith


### Q12: Find the facilities with their usage by member, but not guests

In [13]:
# Retrieve facilities usage by members
cursor.execute("SELECT Facilities.name AS facility_name, COUNT(Bookings.bookid) AS member_usage FROM Bookings JOIN Facilities ON Bookings.facid = Facilities.facid WHERE Bookings.memid != 0 GROUP BY Facilities.name;")
df = pd.DataFrame(cursor.fetchall())

#Display column names
df.columns = [description[0] for description in cursor.description]

# Display the results
print(df.head())



     facility_name  member_usage
0  Badminton Court           344
1   Massage Room 1           421
2   Massage Room 2            27
3       Pool Table           783
4    Snooker Table           421


### Q13: Find the facilities usage by month, but not guests

In [14]:
# Retrieve facilities usage by month for members
cursor.execute("SELECT Facilities.name AS facility_name, strftime('%Y-%m', Bookings.starttime) AS month, COUNT(Bookings.bookid) AS usage_count FROM Bookings JOIN Facilities ON Bookings.facid = Facilities.facid WHERE Bookings.memid != 0 GROUP BY Facilities.name, month;")
df = pd.DataFrame(cursor.fetchall())

#Display column names
df.columns = [description[0] for description in cursor.description]

# Display the results
print(df.head())

connection.close()

     facility_name    month  usage_count
0  Badminton Court  2012-07           51
1  Badminton Court  2012-08          132
2  Badminton Court  2012-09          161
3   Massage Room 1  2012-07           77
4   Massage Room 1  2012-08          153
